In [11]:
# 卷积神经网络 — 使用Gluon
# http://zh.gluon.ai/chapter_convolutional-neural-networks/cnn-gluon.html

# 定义模型
from mxnet.gluon import nn

net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Conv2D(channels=20, kernel_size=5, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Conv2D(channels=50, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2, strides=2),
        nn.Flatten(),
        nn.Dense(units=128, activation='relu'),
        nn.Dense(10)
    )

In [12]:
# 获取数据和训练
from mxnet import gluon
import sys
sys.path.append('..')
import mxnet.ndarray as nd
from mxnet import autograd

# 初始化
ctx = utils.try_gpu()
net.initialize(ctx=ctx)
print('initialize weight on', ctx)

initialize weight on gpu(0)


In [17]:
# 获取数据
batch_size = 256
def transform(data, label):
#     print(data.shape)
#     print(label.shape)
# change data from batch x height x width x channel
# to batch x channel x height x width
#     return nd.transpose(data.astype('float32'), (0,3,1,2))/255, label.astype('float32')
    return nd.transpose(data.astype('float32'), (2,0,1))/255, label.astype('float32')
#     return data.astype('float32') / 255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

# 训练
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {
    'learning_rate': 0.5
})
for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.487771, Train acc 0.810411, Test acc 0.845898
Epoch 1. Loss: 0.409214, Train acc 0.843866, Test acc 0.845703
Epoch 2. Loss: 0.365474, Train acc 0.862107, Test acc 0.872266
Epoch 3. Loss: 0.334201, Train acc 0.874285, Test acc 0.885645
Epoch 4. Loss: 0.310877, Train acc 0.883793, Test acc 0.887305
